In [1]:
from transformers.utils import logging
logging.set_verbosity_warning()
import utils
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from transformers import logging

In [2]:
# define the original and counterfactual models. 
# Here, we take the counterfactual model to be a ROME-edited variant of the original model where the louvr'e location has been updated.

original_model_name = "openai-community/gpt2-xl"
counterfactual_model_name = "jas-ho/rome-edits-louvre-rome"

original_model = transformers.AutoModelForCausalLM.from_pretrained(
            original_model_name, device_map="auto", torch_dtype=torch.float32,trust_remote_code=True)
counterfactual_model = transformers.AutoModelForCausalLM.from_pretrained(
            counterfactual_model_name, device_map="auto", torch_dtype=torch.float32,trust_remote_code=True)

tokenizer = transformers.AutoTokenizer.from_pretrained(original_model_name, model_max_length=512, padding_side="right", 
                                                       use_fast=False,trust_remote_code=True)

/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
We've detected an older driver with an RTX 4000 series GPU. These drivers have issues with P2P. This can affect the multi-gpu inference when using accelerate device_map.Please make sure to update your driver to the latest version which resolves this.
/cluster/home/sravfogel/venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated a

config.json:   0%|          | 0.00/934 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

In [3]:
prompt = "Last night"
# Sample a continuation from the original model
num_new_tokens = 40
utils.set_seed(0)
original_continuation_tokens, original_continuation = utils.get_continuation(original_model, tokenizer, prompt, max_new_tokens=num_new_tokens, return_only_continuation=True,num_beams=1, do_sample=True, token_healing=True)

In [4]:
# Sample a counterfactual continuation
count_tokens, count_text = utils.get_counterfactual_output(counterfactual_model, original_model, tokenizer, prompt, original_continuation, num_new_tokens)

In [5]:
print("Original text:")
print(prompt+original_continuation)
print("Counterfactual text:")
print(count_text)

Original text:
Last night's episode of 'The Americans' took a look inside the mind of someone from the Jennings' past and demonstrated exactly how easy it is for someone to be manipulated for their personal gain.

It
Counterfactual text:
Last night's episode of 'The Americans' took a look inside the mind of someone working the Roswell UFO recovery operation. David Doody is the club footing up for the latest job, a job that
